# YAPILACAKLAR

* Edge detected görselleri diff görsel ile karşılaştır.
* En iyi (uyumlu) edge detection bulmaya çalış
* SREdgeNet araştırılabilir
* SRGAN çalışmasında PSNR ve SSIM karşılaştırmaları neden Y kanalı üzerinde gerçekleştiriliyor?
* DAHA DA ÖNEMLİSİ: Tüm renk kanalları üzerinde aynı anda PSNR ve SSIM hesaplamak güvenilir mi? Çünkü her renk kanalının kendi içinde max. ve min. değerleri farklı, oysa PSNR formülünde önemli max. value kullanılıyor, bu önemli bir detay! PSNR hesaplaması RGB için bir farka sebep olmazken YCC ve CIELAB renk uzaylarında önemli bir fark yaratabilir. Bunun da karşılaştırmalı sonuçlar bir yayın olabilir.
* INTERPOLASYON yöntemleri harici bir metot olarak kullanılabilmeli

* clip() ile maximum değeri aşan INTERPOLASYON yöntemlerinde limitleme yapıldı. - clip() iptal edildi ve Normalizasyon işlemi uygulandı.
* Tüm renk uzaylarının olası renk adetlerini listele, az çeşide indirgenen renk uzaylarında başarının kediliğinden yükseldiği hipotezini doğrulamaya çalış
* normalize_values() FONKSİYONUNDAKİ NOTU OKU!!!
* normalize_factor hep 255 olarak alınıyor. Diğer renk uzaylarının eksenlerine göre değişiklik göstermesi gerekmez mi?
* Fark hesaplamasında elde edilen fark rgb2gray() ile dönüştürülüyor, ancak CMYK ile de işlem yapıyoruz. Bunun düzeltilmesi gerekli!
* modified_dir neden yapıldı?

# ÇÖZÜNÜRLÜK İYİLEŞTİRME

## Tanımlamalar

In [1]:
enhancement_mode=False
edge_detection_mode=False
show_comparison_results=True
# Results of outputs' modified versions, output_modified_dir[] will be used instead of
# output_dir[]
modified_results=False

import cv2
from enum import Enum


class Conversions(Enum):
  NONE = 0
  RGB2YCRCB = 1
  RGB2LAB = 2
  #RGB2CMYK = 3
  #BGR2YCRCB = 2

class Methods(Enum):
  INTERPOLATION_NN = 0
  INTERPOLATION_LINEAR =1
  INTERPOLATION_CUBIC =2
  INTERPOLATION_LANCZOS =3
  SRGAN = 4
  REAL_ESRGAN = 5
  SWIN2SR = 6

class Dataset():
  def __init__(self, name, path, lr_dir, hr_dir):
    self.name = name
    self.path = path
    self.lr_dir = path + lr_dir +"/"
    self.hr_dir = path + hr_dir + "/"

DATASET_LIST = {}
DATASET_LIST["Set5"] = Dataset("Set5","/content/drive/MyDrive/Colab Notebooks/SRGAN/dataset/Set5/","LRbicx4","GTmod12")
DATASET_LIST["Set14"] = Dataset("Set14","/content/drive/MyDrive/Colab Notebooks/SRGAN/dataset/Set14/","LRbicx4","GTmod12")
DATASET_LIST["BSDS100"] = Dataset("BSDS100","/content/drive/MyDrive/Colab Notebooks/SRGAN/dataset/BSDS100/","LR","HR")
DATASET_LIST["URBAN100"] = Dataset("URBAN100","/content/drive/MyDrive/Colab Notebooks/SRGAN/dataset/URBAN100/","LR","HR")


#Use this:
dataset = DATASET_LIST["URBAN100"]
print(dataset.name +" is default dataset.")

input_dir = dataset.lr_dir
input_lab_dir = input_dir + "lab/"
input_edge_dir = input_dir + "edge/"
ground_truth_dir = dataset.hr_dir

base_dir={}
output_dir={}
output_diff_dir={}
output_modified_dir={}


base_dir[Methods.INTERPOLATION_NN]="/content/drive/MyDrive/Colab Notebooks/SRGAN/0-Interpolations/NN/"
output_dir[Methods.INTERPOLATION_NN]=base_dir[Methods.INTERPOLATION_NN]+"results/"+dataset.name+"/"
output_diff_dir[Methods.INTERPOLATION_NN]= output_dir[Methods.INTERPOLATION_NN] + "diff/"

base_dir[Methods.INTERPOLATION_LINEAR]="/content/drive/MyDrive/Colab Notebooks/SRGAN/0-Interpolations/LINEAR/"
output_dir[Methods.INTERPOLATION_LINEAR]=base_dir[Methods.INTERPOLATION_LINEAR]+"results/"+dataset.name+"/"
output_diff_dir[Methods.INTERPOLATION_LINEAR]= output_dir[Methods.INTERPOLATION_LINEAR] + "diff/"

base_dir[Methods.INTERPOLATION_CUBIC]="/content/drive/MyDrive/Colab Notebooks/SRGAN/0-Interpolations/CUBIC/"
output_dir[Methods.INTERPOLATION_CUBIC]=base_dir[Methods.INTERPOLATION_CUBIC]+"results/"+dataset.name+"/"
output_diff_dir[Methods.INTERPOLATION_CUBIC]= output_dir[Methods.INTERPOLATION_CUBIC] + "diff/"

base_dir[Methods.INTERPOLATION_LANCZOS]="/content/drive/MyDrive/Colab Notebooks/SRGAN/0-Interpolations/LANCZOS/"
output_dir[Methods.INTERPOLATION_LANCZOS]=base_dir[Methods.INTERPOLATION_LANCZOS]+"results/"+dataset.name+"/"
output_diff_dir[Methods.INTERPOLATION_LANCZOS]= output_dir[Methods.INTERPOLATION_LANCZOS] + "diff/"

base_dir[Methods.SRGAN]="/content/drive/MyDrive/Colab Notebooks/SRGAN/5-SRGAN-PyTorch-master/"
output_dir[Methods.SRGAN]=base_dir[Methods.SRGAN]+"results/test/v1/"+dataset.name+"/"
output_diff_dir[Methods.SRGAN]= output_dir[Methods.SRGAN] + "diff/"
output_modified_dir[Methods.SRGAN]=output_dir[Methods.SRGAN] + "modified/"

base_dir[Methods.REAL_ESRGAN]="/content/drive/MyDrive/Colab Notebooks/SRGAN/6-Real-ESRGAN/"
output_dir[Methods.REAL_ESRGAN]=base_dir[Methods.REAL_ESRGAN]+"results/"+dataset.name+"/"
output_diff_dir[Methods.REAL_ESRGAN]= output_dir[Methods.REAL_ESRGAN] + "diff/"

base_dir[Methods.SWIN2SR]="/content/drive/MyDrive/Colab Notebooks/SRGAN/9-Swin2SR/"
output_dir[Methods.SWIN2SR]=base_dir[Methods.SWIN2SR]+"results/swin2sr_classical_sr_x4/"+dataset.name+"/"
output_diff_dir[Methods.SWIN2SR]= output_dir[Methods.SWIN2SR] + "diff/"



URBAN100 is default dataset.


In [2]:
class ColorAxis():
  def __init__(self,name,min,max):
    self.name=name
    self.min=min
    self.max=max


class Color():
  def __init__(self, name, axes):
    self.name = name
    self.axes = axes

COLORS = {}
COLORS["RGB"] = Color("RGB",[
    ColorAxis("R",0,255),
    ColorAxis("G",0,255),
    ColorAxis("B",0,255)
])

COLORS["YCC"] = Color("YCC",[
    ColorAxis("Y",16,235),
    ColorAxis("Cb",16,240),
    ColorAxis("Cr",16,240)
])

COLORS["LAB"] = Color("LAB",[
    ColorAxis("L",0,100),
    ColorAxis("A",-110,110),
    ColorAxis("B",-110,110)
])

COLORS["CMYK"] = Color("CMYK",[
    ColorAxis("C",0,100),
    ColorAxis("M",0,100),
    ColorAxis("Y",0,100),
    ColorAxis("K",0,100)
])



In [ ]:
#@title Dosya Sistemine Erişim

#https://github.com/Lornatang/SRGAN-PyTorch

import os
from google.colab import drive
drive.mount('/content/drive')


# Add Tools
import sys
#sys.path.insert(0, '/content/drive/My Drive/Colab Notebooks/Tools')
#from Compare import *

import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

print(device)

## Araçlar

In [ ]:
#@title Dosya Sistemi Yönetimi
from os import walk

def get_files(directory):
  return next(walk(directory), (None, None, []))[2]  # [] if no file

def create_directory_if_not_exists(directory):
  if not os.path.exists(directory):
    os.makedirs(directory)

def file_to_matrix(file_src,conversion,remove_alpha=True,normalize_factor=1.0):
  # Both of the same

  #import matplotlib.image as mpimg
  #img = np.array(img_to_array(load_img(file_src))).astype(np.float32) / 255.0

  #print("[file_to_matrix()]"+file_src)

  if (".npy" in file_src):
    img = np.load(file_src,allow_pickle=False)
    #img = img / normalize_factor
  else:
    #RGB format
    #img = cv2.imread(file_src, cv2.IMREAD_UNCHANGED).astype(np.float32) / normalize_factor
    img = cv2.imread(file_src, cv2.IMREAD_COLOR).astype(np.float32)  / normalize_factor

    # BGR -> RGB
    img = img[...,::-1]

  #Removing alpha channel if exists
  if remove_alpha:
    img = remove_alpha_channel(img)

  #show_stats(img)
  img,color=color_convert(img,conversion)

  return img,color

def remove_alpha_channel(img):
  img = img[:,:,0:3]
  return img

### Renk Uzayı Dönüşümü

In [ ]:
def color_convert(img,conversion):
  if (conversion==Conversions.RGB2YCRCB):
    return rgb2ycrcb(img),COLORS["YCC"]
  elif (conversion==Conversions.RGB2LAB):
    return rgb2lab(img),COLORS["LAB"]
  elif (conversion == Conversions.NONE):
    return img,COLORS["RGB"]
  elif (conversion==Conversions.RGB2CMYK):
    return rgb2cmyk(img),COLORS["CMYK"]
  else:
    raise Exception("Böyle bir dönüşüm tanımlı değil: "+conversion.name)

In [ ]:
#@title RGB2GRAY
def rgb2gray(matrix):
    return np.dot(matrix[...,:3], [0.2989, 0.5870, 0.1140])

In [ ]:
#@title RGB2CMYK

def rgb2cmyk_old(img):
  img = Image.fromarray(np.uint8(img*255)).convert('CMYK')
  img = np.asarray(img)
  #show_stats(img)
  return img


RGB_SCALE = 255
CMYK_SCALE = 100

def rgb_to_cmyk(r, g, b):
  if (r, g, b) == (0, 0, 0):
    # black
    return 0, 0, 0, CMYK_SCALE

  # rgb [0,255] -> cmy [0,1]
  c = 1 - r / RGB_SCALE
  m = 1 - g / RGB_SCALE
  y = 1 - b / RGB_SCALE

  # extract out k [0, 1]
  min_cmy = min(c, m, y)
  c = (c - min_cmy) / (1 - min_cmy)
  m = (m - min_cmy) / (1 - min_cmy)
  y = (y - min_cmy) / (1 - min_cmy)
  k = min_cmy

  # rescale to the range [0,CMYK_SCALE]
  return c * CMYK_SCALE, m * CMYK_SCALE, y * CMYK_SCALE, k * CMYK_SCALE


# Daha iyi bir yaklaşım olabilir, doğruluğu test edilmedi.
def rgb2cmyk_check(img):
  from PIL import Image
  return Image.fromarray(img).convert('CMYK')

def rgb2cmyk(img):
  # Create float
  #print("converting rgb 2 cmyk")
  '''
  print("rgb")
  show_stats(img)
  '''
  rgb = img
  bgr = rgb[...,::-1]
  '''
  print("bgr")
  show_stats(bgr)
  '''

  # Extract channels
  with np.errstate(invalid='ignore', divide='ignore'):
    K = 1 - np.max(bgr, axis=2)
    C = (1-bgr[...,2] - K)/(1-K)
    M = (1-bgr[...,1] - K)/(1-K)
    Y = (1-bgr[...,0] - K)/(1-K)

  # Convert the input BGR image to CMYK colorspace
  CMYK = (np.dstack((C,M,Y,K)) * 255).astype(np.uint8)
  '''
  print("c"*20)
  show_stats(CMYK[...,0])
  print("m"*20)
  show_stats(CMYK[...,1])
  print("y"*20)
  show_stats(CMYK[...,2])
  print("k"*20)
  show_stats(CMYK[...,3])
  '''
  return CMYK

In [ ]:
#@title RGB2LAB
#https://stackoverflow.com/questions/40586276/unexpected-output-while-converting-rgb-image-to-lab-image
def bgr2lab(img):
  return cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

def rgb2lab(img):

  #print("rgb2lab() Before Conversion")
  #show_stats(img)
  #show_stats(img)
  img = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
  #print("rgb2lab() After Conversion")
  #show_stats(img)

  #https://towardsdatascience.com/computer-vision-101-working-with-color-images-in-python-7b57381a8a54
  img = (img + [0, 128, 128]) / [100, 255, 255]

  '''
  ax = img
  fig, ax = plt.subplots(1, 4, figsize = (18, 30))
  ax[0].imshow(img)
  ax[0].axis('off')
  ax[0].set_title('Lab')
  for i, col in enumerate(['L', 'a', 'b'], 1):
      cv2_imshow(img[:, :, i-1], ax=ax[i])
      ax[i].axis('off')
      ax[i].set_title(col)
  fig.show()
  '''

  return img

  #return cv2.cvtColor(img, cv2.COLOR_RGB2LAB)

def cielab_conversion_test_1(org):
  import cv2
  from google.colab.patches import cv2_imshow
  import numpy as np
  #org = np.float32(cv2.imread(input_dir+"bird.png", -1))/255.
  #print(org.shape)
  lab_image = cv2.cvtColor(org, cv2.COLOR_BGR2LAB)
  l, a, b = cv2.split(lab_image)
  l_scaled = np.uint8(255.*(l - l.min())/(l.max() - l.min()))
  a_scaled = np.uint8(255.*(a - a.min())/(a.max() - a.min()))
  b_scaled = np.uint8(255.*(b - b.min())/(b.max() - b.min()))

  cv2_imshow(l)
  cv2_imshow(a)
  cv2_imshow(b)

  cv2_imshow(l_scaled)
  cv2_imshow(a_scaled)
  cv2_imshow(b_scaled)

def cielab_conversion_test_2():
  from skimage import io, color
  import matplotlib.pyplot as plt

  org = io.imread(input_dir+"bird.png")
  lab_image = color.rgb2lab(org)
  l = lab_image[:, :, 0]
  a = lab_image[:, :, 1]
  b = lab_image[:, :, 2]

  fig, ax = plt.subplots(1, 3)
  plt.set_cmap('gray')

  ax[0].imshow(l)
  ax[1].imshow(a)
  ax[2].imshow(b)

#cielab_conversion_test_1()
#cielab_conversion_test_2()

In [ ]:
# Genelleştir
def rgb2lab_batch():
  import numpy as np
  import cv2

  create_directory_if_not_exists(input_lab_dir)

  file_names = get_files(input_dir)
  for file_name in file_names:
    img = cv2.imread(input_dir + file_name)
    LAB = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    #cv2.imwrite('L.png', LAB[:,:,0])
    #cv2.imwrite('a.png', LAB[:,:,1])
    #cv2.imwrite('b.png', LAB[:,:,2])

    # BGR = cv2.cvtColor(LAB, cv2.COLOR_LAB2BGR)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = cv2.imwrite(input_lab_dir + file_name, LAB)
    print(file_name + " :" + str(result))

In [ ]:
#@title RGB2YCrCb and BGR2YCrCb
def rgb2ycrcb(img):
  return cv2.cvtColor(img, cv2.COLOR_RGB2YCR_CB)

def bgr2ycrcb(img):
  return cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)

def getYfromRGB(img):
  return rgb2ycrcb(img)[:,:,0]

### Interpolasyon Yöntemleri

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt, floor, ceil
from PIL import Image
import math


def read_image(path):
    '''Read image and return the image propertis.
    Parameters:
    path (string): Image path
    Returns:
    numpy.ndarray: Image exists in "path"
    list: Image size
    tuple: Image dimension (number of rows and columns)
    '''
    print(path)
    img = cv2.imread(path)  # cv2.IMREAD_GRAYSCALE)
    size = img.shape
    dimension = (size[0], size[1])

    return img, size, dimension


def image_change_scale(img, dimension, scale=100, interpolation=cv2.INTER_LINEAR):
    '''Resize image to a specificall scale of original image.
    Parameters:
    img (numpy.ndarray): Original image
    dimension (tuple): Original image dimension
    scale (int): Multiply the size of the original image
    Returns:
    numpy.ndarray: Resized image
    '''
    scale /= 100
    new_dimension = (int(dimension[1]*scale), int(dimension[0]*scale))
    resized_img = cv2.resize(img, new_dimension, interpolation=interpolation)

    return resized_img


def nearest_interpolation(image, factor):
    '''Nearest neighbor interpolation method to convert small image to original image
    Parameters:
    img (numpy.ndarray): Small image
    factor: enlargement
    Returns:
    numpy.ndarray: Resized image
    '''
    width = image.shape[0] * factor
    height = image.shape[1] * factor

    new_image = np.zeros((width, height, image.shape[2]))

    for i in range(width):
        for j in range(height):
            row = floor(i / factor)
            column = floor(j / factor)

            new_image[i, j] = image[row, column]

    return new_image


def bilinear_interpolation(image, factor):
    '''Bilinear interpolation method to convert small image to original image
    Parameters:
    img (numpy.ndarray): Small image
    dimension (tuple): resizing image dimension
    Returns:
    numpy.ndarray: Resized image
    '''
    height = image.shape[0]
    width = image.shape[1]

    scale_x = (width)/(image.shape[1] * factor)
    scale_y = (height)/(image.shape[0] * factor)

    new_image = np.zeros((image.shape[0] * factor, image.shape[1] * factor, image.shape[2]))

    for k in range(3):
        for i in range(image.shape[0] * factor):
            for j in range(image.shape[1] * factor):
                x = (j+0.5) * (scale_x) - 0.5
                y = (i+0.5) * (scale_y) - 0.5

                x_int = int(x)
                y_int = int(y)

                # Prevent crossing
                x_int = min(x_int, width-2)
                y_int = min(y_int, height-2)

                x_diff = x - x_int
                y_diff = y - y_int

                a = image[y_int, x_int, k]
                b = image[y_int, x_int+1, k]
                c = image[y_int+1, x_int, k]
                d = image[y_int+1, x_int+1, k]

                pixel = a*(1-x_diff)*(1-y_diff) + b*(x_diff) * \
                    (1-y_diff) + c*(1-x_diff) * (y_diff) + d*x_diff*y_diff

                new_image[i, j, k] = pixel.astype(np.uint8)

    return new_image


def W(x):
    '''Weight function that return weight for each distance point
    Parameters:
    x (float): Distance from destination point
    Returns:
    float: Weight
    '''
    a = -0.5
    pos_x = abs(x)
    if -1 <= abs(x) <= 1:
        return ((a+2)*(pos_x**3)) - ((a+3)*(pos_x**2)) + 1
    elif 1 < abs(x) < 2 or -2 < x < -1:
        return ((a * (pos_x**3)) - (5*a*(pos_x**2)) + (8 * a * pos_x) - 4*a)
    else:
        return 0


def bicubic_interpolation(img, factor):
    '''Bicubic interpolation method to convert small size image to original size image
    Parameters:
    img (numpy.ndarray): Small image
    dimension (tuple): resizing image dimension
    Returns:
    numpy.ndarray: Resized image
    '''


    nrows = img.shape[0]*factor
    ncols = img.shape[1]*factor

    output = np.zeros((nrows, ncols, img.shape[2]), np.uint8)
    for c in range(img.shape[2]):
        for i in range(nrows):
            for j in range(ncols):
                xm = (i + 0.5) * (img.shape[0]/(img.shape[0]*factor)) - 0.5
                ym = (j + 0.5) * (img.shape[1]/(img.shape[1]*factor)) - 0.5

                xi = floor(xm)
                yi = floor(ym)

                u = xm - xi
                v = ym - yi

                # -------------- Using this make ignore some points and increase the value of black in image border
                # x = [(xi - 1), xi, (xi + 1), (xi + 2)]
                # y = [(yi - 1), yi, (yi + 1), (yi + 2)]
                # if ((x[0] >= 0) and (x[3] < img.shape[1]) and (y[0] >= 0) and (y[3] < img.shape[0])):
                #     dist_x0 = W(x[0] - xm)
                #     dist_x1 = W(x[1] - xm)
                #     dist_x2 = W(x[2] - xm)
                #     dist_x3 = W(x[3] - xm)
                #     dist_y0 = W(y[0] - ym)
                #     dist_y1 = W(y[1] - ym)
                #     dist_y2 = W(y[2] - ym)
                #     dist_y3 = W(y[3] - ym)

                #     out = (img[x[0], y[0], c] * (dist_x0 * dist_y0) +
                #            img[x[0], y[1], c] * (dist_x0 * dist_y1) +
                #            img[x[0], y[2], c] * (dist_x0 * dist_y2) +
                #            img[x[0], y[3], c] * (dist_x0 * dist_y3) +
                #            img[x[1], y[0], c] * (dist_x1 * dist_y0) +
                #            img[x[1], y[1], c] * (dist_x1 * dist_y1) +
                #            img[x[1], y[2], c] * (dist_x1 * dist_y2) +
                #            img[x[1], y[3], c] * (dist_x1 * dist_y3) +
                #            img[x[2], y[0], c] * (dist_x2 * dist_y0) +
                #            img[x[2], y[1], c] * (dist_x2 * dist_y1) +
                #            img[x[2], y[2], c] * (dist_x2 * dist_y2) +
                #            img[x[2], y[3], c] * (dist_x2 * dist_y3) +
                #            img[x[3], y[0], c] * (dist_x3 * dist_y0) +
                #            img[x[3], y[1], c] * (dist_x3 * dist_y1) +
                #            img[x[3], y[2], c] * (dist_x3 * dist_y2) +
                #            img[x[3], y[3], c] * (dist_x3 * dist_y3))

                #     output[i, j, c] = np.clip(out, 0, 255)
                # ---------------------------

                out = 0
                for n in range(-1, 3):
                    for m in range(-1, 3):
                        if ((xi + n < 0) or (xi + n >= img.shape[1]) or (yi + m < 0) or (yi + m >= img.shape[0])):
                            continue

                        out += (img[xi+n, yi+m, c] * (W(u - n) * W(v - m)))

                output[i, j, c] = np.clip(out, 0, 255)

    return output


def error_calculator(img1, img2):
    '''Calculate the average difference between the image pixels and returns a number.
    Parameters:
    image (numpy.ndarray): Interpolated image
    image (numpy.ndarray): Original image
    Returns:
    float: Average difference between two images
    '''
    return np.average(abs(np.array(img1, dtype="int16") -
                          np.array(img2, dtype="int16")))


def error_calculator_manual(img1, img2):
    '''Calculate the average difference between the image pixels and returns a number.
    Parameters:
    image (numpy.ndarray): Interpolated image
    image (numpy.ndarray): Original image
    Returns:
    float: Average difference between two images
    '''
    result = 0

    img1 = np.array(img1, dtype="int16")
    img2 = np.array(img2, dtype="int16")

    for i, j in zip(img1, img2):
        for k, l in zip(i, j):
            for v in range(len(k)):
                result += abs(k[v] - l[v])

    if len(img1.shape) == 3:
        return result/(img1.shape[0] * img1.shape[1] * img1.shape[2])
    else:
        return result/(img1.shape[0] * img1.shape[1])


def show_result(images_list):
    '''Show result of each image action.
    Parameters:
    images list (list): List of all images
    Returns
    '''
    titles = list(images_list.keys())
    images = list(images_list.values())

    fig, axs = plt.subplots(2, 3)
    fig.suptitle('25 Percent of the original size', fontsize=16)

    axs[0, 0].set_title(titles[0])
    axs[0, 0].imshow(cv2.cvtColor(images[0], cv2.COLOR_BGR2RGB))

    axs[0, 1].set_title(titles[1])
    axs[0, 1].imshow(cv2.cvtColor(images[1], cv2.COLOR_BGR2RGB))

    axs[0, 2].set_title(titles[2])
    axs[0, 2].imshow(cv2.cvtColor(images[2], cv2.COLOR_BGR2RGB))

    axs[1, 0].set_title(titles[3])
    axs[1, 0].imshow(cv2.cvtColor(images[3], cv2.COLOR_BGR2RGB))

    axs[1, 1].set_title(titles[4])
    axs[1, 1].imshow(cv2.cvtColor(images[4], cv2.COLOR_BGR2RGB))

    axs[1, 2].set_title(titles[5])
    axs[1, 2].imshow(cv2.cvtColor(images[5], cv2.COLOR_BGR2RGB))


def result_comparison(error_list, color):
    '''Error rate comparison between the different interpolation methods.
    Parameters:
    error (list): Error rates
    Returns
    '''
    interpolation_methods = ["Nearest Opencv", "Nearest Neighbor",
                             "Bilinear Opencv", "Bilinear", "Cubiclinear Opencv", "Cubiclinear", "Lanczos"]

    print("\n........................Error calculate between the smalled image and the original image............................\n")
    print(f"{interpolation_methods[0]} Error Rate: {error_list[0]}")
    print(f"{interpolation_methods[1]} Error Rate: {error_list[1]}")
    print(f"{interpolation_methods[2]} Error Rate: {error_list[2]}")
    print(f"{interpolation_methods[3]} Error Rate: {error_list[3]}")
    print(f"{interpolation_methods[4]} Error Rate: {error_list[4]}")
    print(f"{interpolation_methods[5]} Error Rate: {error_list[5]}")
    print(f"{interpolation_methods[6]} Error Rate: {error_list[6]}\n")

    plt.figure()
    plt.bar(interpolation_methods, error_list, color=[
            'red', 'blue', 'purple', 'green', 'fuchsia', 'yellow', 'black'])
    plt.title("Compare result between smalled and original size image")




In [ ]:
def main():
    images_list = {}

    sample_data = ground_truth_dir + "bird.png"
    # Read Image
    #img, size, dimension = read_image("./butterfly.png")
    img, size, dimension = read_image(sample_data)
    print(f"Image size is: {size}")
    images_list['Original Image'] = img

    # Change Image Size
    scale_percent = 25  # percent of original image size
    resized_img = image_change_scale(img, dimension, scale_percent)
    print(f"Smalled Image size is: {resized_img.shape}")
    images_list['Smalled Image'] = resized_img

    fig, axs = plt.subplots(2, 2)
    fig.suptitle('My Implementation', fontsize=10)

    # Change image to original size using nearest neighbor interpolation
    nn_img = image_change_scale(resized_img, dimension, interpolation=cv2.INTER_NEAREST)
    images_list['Nearest Neighbor Interpolation'] = nn_img

    nn_img_algo = nearest_interpolation(resized_img, 4)
    nn_img_algo = Image.fromarray(nn_img_algo.astype('uint8')).convert('RGB')

    # Change image to original size using bilinear interpolation
    bil_img = image_change_scale(
        resized_img, dimension, interpolation=cv2.INTER_LINEAR)
    images_list['Bilinear Interpolation'] = bil_img

    bil_img_algo = bilinear_interpolation(resized_img, 4)
    bil_img_algo = Image.fromarray(bil_img_algo.astype('uint8')).convert('RGB')

    # Change image to original size using cubiclinear interpolation (4*4 pixel neighborhood)
    cubic_img = image_change_scale(
        resized_img, dimension, interpolation=cv2.INTER_CUBIC)
    images_list['CubicLinear Interpolation'] = cubic_img

    # cubic_img_algo = BiCubic_interpolation(
    #     resized_img, dimension[0], dimension[1])
    cubic_img_algo = bicubic_interpolation(resized_img, 4)
    cubic_img_algo = Image.fromarray(
        cubic_img_algo.astype('uint8')).convert('RGB')

    # Change image to original size using lanczos interpolation (8*8 pixel neighborhood)
    czos_img = image_change_scale(
        resized_img, dimension, interpolation=cv2.INTER_LANCZOS4)
    images_list['Lanczos Interpolation'] = czos_img

    # error calculate between the smalled image and the original image
    error_list = []
    error_list.append(error_calculator_manual(nn_img, img))
    error_list.append(error_calculator_manual(nn_img_algo, img))
    error_list.append(error_calculator_manual(bil_img, img))
    error_list.append(error_calculator_manual(bil_img_algo, img))
    error_list.append(error_calculator_manual(cubic_img, img))
    error_list.append(error_calculator_manual(cubic_img_algo, img))
    error_list.append(error_calculator_manual(czos_img, img))

    # Show Result
    show_result(images_list)

    # Result Comparison
    result_comparison(error_list, "blue")

    axs[0, 0].set_title("Original")
    axs[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    axs[0, 1].set_title("Nearest")
    axs[0, 1].imshow(cv2.cvtColor(np.array(nn_img_algo), cv2.COLOR_BGR2RGB))

    axs[1, 0].set_title("Bilinear")
    axs[1, 0].imshow(cv2.cvtColor(np.array(bil_img_algo), cv2.COLOR_BGR2RGB))

    axs[1, 1].set_title("Bicubic")
    axs[1, 1].imshow(cv2.cvtColor(np.array(cubic_img_algo), cv2.COLOR_BGR2RGB))

    # plt.grid()

    plt.show()

    cv2.waitKey(0)
    cv2.destroyAllWindows()


'''
if __name__ == "__main__":
    main()
'''

### Kalite Ölçümleri

In [ ]:
#@title PSNR, MSE, SSIM

def psnr(target, ref, conversion):
    # Assume target is RGB/BGR image
    target_data = target.astype(np.float32)
    ref_data = ref.astype(np.float32)


    #!!! CHANGED - Uncomment
    target_data = target
    ref_data = ref

    diff = ref_data - target_data
    diff = diff.flatten('C')

    rmse = np.sqrt(np.mean(diff ** 2.))

    # Makale buna göre mi yapıyor?
    # 10. * torch.log10(1. / torch.mean((sr_y_tensor - hr_y_tensor) ** 2))

    return 20 * np.log10(255. / rmse)

def psnr_other(target, ref):

    from math import sqrt
    mse = np.mean((target - ref) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / sqrt(mse))
    return psnr

def mse(target, ref):
    target_data = target.astype(np.float32)
    ref_data = ref.astype(np.float32)
    err = np.sum((target_data - ref_data) ** 2)

    err /= float(target_data.shape[0] * target_data.shape[1])
    return err

from skimage.metrics import structural_similarity as ssim

In [ ]:
#@title A-DISTS
import sys
sys.path.insert(0, '/content/drive/My Drive/Colab Notebooks/SRGAN/7-A-DISTS')
%cd '/content/drive/My Drive/Colab Notebooks/SRGAN/7-A-DISTS'
from ADISTS_pt import *
from torchvision import models,transforms

def prepare_image(image, resize=True):
    if resize and min(image.size)>256:
        image = transforms.functional.resize(image,256)
    image = transforms.ToTensor()(image)
    return image.unsqueeze(0)

def adists(image_ref,image_dist):
  '''
  image_ref,_=file_to_matrix("images/r0.png",conversion=Conversions.NONE)
  image_dist,_=file_to_matrix("images/r1.png",conversion=Conversions.NONE)
  # score: 0.3347
  '''
  image_ref=image_ref*255.
  image_dist=image_dist*255.

  image_ref = Image.fromarray(image_ref.astype('uint8'), 'RGB')
  image_dist = Image.fromarray(image_dist.astype('uint8'), 'RGB')

  ref = prepare_image(image_ref,False)
  dist = prepare_image(image_dist,False)
  assert ref.shape == dist.shape

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = ADISTS().to(device)
  ref = ref.to(device)
  dist = dist.to(device)
  score = model(ref, dist)

  return 1.0*score.item()


/content/drive/My Drive/Colab Notebooks/SRGAN/7-A-DISTS


In [ ]:
#@title DISTS
sys.path.insert(1, '/content/drive/My Drive/Colab Notebooks/SRGAN/7-DISTS-IQA')
%cd '/content/drive/My Drive/Colab Notebooks/SRGAN/7-DISTS-IQA'
from DISTS_pytorch import *
from torchvision import models,transforms

def prepare_image(image, resize=True):
    if resize and min(image.size)>256:
        image = transforms.functional.resize(image,256)
    image = transforms.ToTensor()(image)
    return image.unsqueeze(0)

def dists(image_ref,image_dist):
  '''
  image_ref and image_dist comes in [0,1] range, RGB
  '''
  image_ref=image_ref*255.
  image_dist=image_dist*255.

  #image_ref,_=file_to_matrix("images/r0.png",conversion=Conversions.NONE)
  #image_dist,_=file_to_matrix("images/r1.png",conversion=Conversions.NONE)
  # score: 0.3347

  image_ref = Image.fromarray(image_ref.astype('uint8'), 'RGB')
  image_dist = Image.fromarray(image_dist.astype('uint8'), 'RGB')

  ref = prepare_image(image_ref,False)
  dist = prepare_image(image_dist,False)
  assert ref.shape == dist.shape

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = DISTS().to(device)
  ref = ref.to(device)
  dist = dist.to(device)
  score = model(ref, dist)

  return 1.0*score.item()


/content/drive/My Drive/Colab Notebooks/SRGAN/7-DISTS-IQA


In [ ]:
#@title LPIPS
!pip install lpips
import lpips


import torch
#!pip install torchmetrics[image]
#_ = torch.manual_seed(123)
#from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
#lpips_calculate = LearnedPerceptualImagePatchSimilarity(net_type='alex')
#lpips_calculate = lpips.LPIPS(net='alex')
lpips_calculate = lpips.LPIPS(net='vgg')

def lpips(img1,img2):
  lpips_output = img1.transpose(2, 0, 1).copy()
  lpips_output = torch.tensor(lpips_output, dtype=torch.float32).unsqueeze(0)
  lpips_output = lpips_output * 2 - 1

  lpips_ground_truth = img2.transpose(2, 0, 1).copy()
  lpips_ground_truth = torch.tensor(lpips_ground_truth, dtype=torch.float32).unsqueeze(0)
  lpips_ground_truth = lpips_ground_truth * 2 - 1


  #img1=normalize_values(img1,-1,1)
  #img2=normalize_values(img2,-1,1)
  #https://torchmetrics.readthedocs.io/en/stable/image/learned_perceptual_image_patch_similarity.html

  # LPIPS needs the images to be in the [-1, 1] range.
  #img1 = (torch.rand(10, 3, 100, 100) * 2) - 1
  #img2 = (torch.rand(10, 3, 100, 100) * 2) - 1
  return lpips_calculate(lpips_output, lpips_ground_truth).item()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.0 MB/s eta 0:00:00
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 239MB/s]


Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/vgg.pth


/usr/local/lib/python3.10/dist-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path, map_location='cpu'), st

In [ ]:
#Görseli kendisi ile karşılaştırınca 0 veriyor!


image_ref,_=file_to_matrix("images/r0-dot-dot.png",conversion=Conversions.NONE)
image_dist,_=file_to_matrix("images/r0-dot.png",conversion=Conversions.NONE)
# score: 0.3347


image_ref = Image.fromarray(image_ref.astype('uint8'), 'RGB')
image_dist = Image.fromarray(image_dist.astype('uint8'), 'RGB')

ref = prepare_image(image_ref,False)
dist = prepare_image(image_dist,False)
assert ref.shape == dist.shape

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DISTS().to(device)
ref = ref.to(device)
dist = dist.to(device)
score = model(ref, dist)

score1 = score.item()
print(score1)


In [ ]:
image_ref,_=file_to_matrix("images/r0.png",conversion=Conversions.NONE)
image_dist,_=file_to_matrix("images/r1.png",conversion=Conversions.NONE)
# score: 0.3347


image_ref = Image.fromarray(image_ref.astype('uint8'), 'RGB')
image_dist = Image.fromarray(image_dist.astype('uint8'), 'RGB')

ref = prepare_image(image_ref,False)
dist = prepare_image(image_dist,False)
assert ref.shape == dist.shape

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ADISTS().to(device)
ref = ref.to(device)
dist = dist.to(device)
score = model(ref, dist)

score1 = score.item()
print(score1)

### Hesaplama ve Çizim Araçları

In [ ]:
from google.colab.patches import cv2_imshow
#from keras.preprocessing.image import  img_to_array,load_img
from tensorflow.keras.utils import img_to_array,load_img
#from QualityMeasurements import *
import json
from datetime import date, datetime, timedelta

class NpEncoder(json.JSONEncoder):
  def default(self, obj):
    if isinstance(obj, np.bool_):
      return bool(obj)
    if isinstance(obj, (np.floating, np.complexfloating)):
      return float(obj)
    if isinstance(obj, np.integer):
      return int(obj)
    if isinstance(obj, np.ndarray):
      return obj.tolist()
    if isinstance(obj, np.string_):
      return str(obj)
    if isinstance(obj, (datetime, date)):
      return obj.isoformat()
    if isinstance(obj, timedelta):
      return str(obj)
    return super(NpEncoder, self).default(obj)


def pretty_print(dictionary):
  json_str = json.dumps(dictionary, cls=NpEncoder,indent=4)
  print(json_str)




def show_stats(img):
  print(img.shape)
  print("MIN : {:10.5f}".format(img.min()))
  print("MAX : {:10.5f}".format(img.max()))
  print("MEAN: {:10.5f}".format(img.mean()))
  print("STD : {:10.5f}".format(img.std()))

def save_image(filename,img):
  plt.imsave(filename, img)
  np.save(filename+".npy",img)
  print(filename+" saved.")

def normalize_values(img,min,max):
  '''
  Görselde hiç BLACK yoksa maximum değer 1 olmuyor, ancak normalize edildiğinde ortaya BLACK çıkıyor.
  Minimum ve maximum değerleri aştığında normalizasyon yapılmalı!!!
  Aksi halde hatalı işlem yapılacak!!!
  '''
  img = (img-img.min()) / (img.max() - img.min())
  return img

def cv2_show_image(image,factor=4,change_order=True):
  #INPUT: RGB
  #cv2 LOVES BGR
  #No need to change grayscale image!
  if (change_order):
    image = image[...,::-1]

  import cv2
  (h,w) = image.shape[:2]
  image = cv2.resize(image,(w * factor,h * factor),cv2.INTER_NEAREST)
  cv2_imshow(image)

def calculate_metrics(output, ground_truth,conversion):
    scores = {}

    #For testing...
    scores["psnr"]=scores["ssim"]=scores["dists"]=scores["adists"]=scores["lpips"]=0

    #show_stats(output)
    #show_stats(ground_truth)
    #print("---")

    scores["psnr"]=psnr(output*255, ground_truth*255, conversion)

    #scores["mse"]=0
    #scores["mse"]=mse(output, ground_truth)

    scores["ssim"]=ssim(output, ground_truth, multichannel=True, channel_axis=2, data_range=1)

    scores["dists"]=dists(output, ground_truth)

    scores["adists"]=adists(output, ground_truth)

    #scores["lpips"]=0
    scores["lpips"]=lpips(output, ground_truth)


    return scores


def show_compared_images(input_src,ground_truth_src,filename,overall_result,conversion,methods,save_diff=True,show_image=False,modified_output=False):
  import matplotlib.pyplot as plt
  import matplotlib.image as mpimg

  # Read images
  #img_input = mpimg.imread(input_src+file)
  #img_ground_truth = mpimg.imread(ground_truth_src+file)

  #Dönüşüm tanımlıysa önce gerçekleşiyor.
  img_input,color_input = file_to_matrix(input_src+filename,conversion,True,255.0)
  img_ground_truth,color_ground_truth = file_to_matrix(ground_truth_src+filename,conversion,True,255.0)



  new_dimension=(img_ground_truth.shape[1],img_ground_truth.shape[0])

  # The same example with just the first two images in a grid 1x2
  figsize_width = 28
  figsize_height = 8
  figsize = (figsize_width, figsize_height)

  #If there's no need to show diff images, put ground truth next to others.
  if (save_diff):
    columns = 1 + len(methods)
    rows=2
  else:
    columns = 2 + len(methods)
    rows=1


  width_ratios = [4]*columns
  width_ratios[0]=1

  if (show_image):
    fig, axes = plt.subplots(nrows=rows,ncols=columns,figsize=figsize,gridspec_kw={'width_ratios': width_ratios})

    if (save_diff):
      axes[0,0].imshow(img_input)
      axes[0,0].set_title("Low Res.")
    else:
      axes[0].imshow(img_input)
      axes[0].set_title("Low Res.")

  i=0

  for output_key in methods:
    #print(output_key)
    #img_output = mpimg.imread(output_dir[output_key]+filename)
    if (modified_output):
      img_output,_ = file_to_matrix(output_modified_dir[output_key]+filename+".npy",conversion,remove_alpha=True,normalize_factor=255.0)
    else:
      img_output,_ = file_to_matrix(output_dir[output_key]+filename+".npy",conversion,remove_alpha=True,normalize_factor=255.0)

    '''
    print("show_compared_images()" + "v"*20)
    show_stats(img_output)
    show_stats(img_ground_truth)
    print("show_compared_images()" + "^"*20)
    '''



    # !!! Convert to grayscale
    if ((img_output.shape[0] != img_ground_truth.shape[0]) or (img_output.shape[1] != img_ground_truth.shape[1])):
      min_row = min(img_output.shape[0],img_ground_truth.shape[0])
      min_col = min(img_output.shape[1],img_ground_truth.shape[1])

      img_ground_truth = img_ground_truth[:min_row,:min_col,:img_ground_truth.shape[2]]
      img_output = img_output[:min_row,:min_col,:img_output.shape[2]]
      print(""+str(output_key)+" shapes are inequal. NEW SHAPE: "+str(img_ground_truth.shape))


    #Euclidean distance or only difference?
    #diff = (img_output**2 + img_ground_truth**2)**(1/2)
    diff = abs(img_output - img_ground_truth)



    #FARK HESAPLAMASI HATALI! HER DÖNÜŞÜM RGB'DEN DÖNÜŞMÜYOR!
    diff_gray = rgb2gray(diff)
    #diff_gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

    #cv2_show_image(diff_gray)
    #cv2_imshow(diff_gray, cmap='gray')

    if (save_diff):
      difference_directory = output_diff_dir[output_key]

      create_directory_if_not_exists(difference_directory)
      #cv2.imwrite(output_diff_dir+output_src.split('/')[-1], diff_gray,cmap='gray')
      plt.imsave(difference_directory+filename, diff_gray,cmap='gray', vmin=0, vmax=1)


    metrics = calculate_metrics(img_output, img_ground_truth, conversion)
    overall_result[output_key.name][conversion.name].append(metrics)
    #overall_result[output_key.value][conversion.value].append(metrics)

    img_new=img_output

    if (show_image):
      if (save_diff):
        set_axis(axes[0,1+i],output_key.name.upper(),img_new,metrics)
        set_axis(axes[1,1+i],"DIFFERENCE",diff_gray)
        axes[1,1+i].imshow(diff_gray, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
      else:
        set_axis(axes[1+i],output_key.name.upper(),img_new,metrics)

    i=i+1

  if (show_image):
    if (save_diff):
      set_axis(axes[1,0],"Ground Truth",img_ground_truth)
    else:
      set_axis(axes[i+1],"Ground Truth",img_ground_truth)
    plt.show()


def set_axis(axis,title,img,metrics=None):
  axis.imshow(img)
  if (metrics is None):
    title_suffix=""
  else:
    #title_suffix = "\nPSNR: {:10.5f}".format(metrics["psnr"])+"\nSSIM: {:10.5f}".format(metrics["ssim"])+"\nLPIPS: {:10.5f}".format(metrics["lpips"])+"\nDISTS: {:10.5f}".format(metrics["dists"])+"\nA-DISTS: {:10.5f}".format(metrics["adists"])
    title_suffix = "\nPSNR: {:10.5f}".format(metrics["psnr"])+"\nSSIM: {:10.5f}".format(metrics["ssim"])+"\nDISTS: {:10.5f}".format(metrics["dists"])+"\nA-DISTS: {:10.5f}".format(metrics["adists"])+"\nLPIPS: {:10.5f}".format(metrics["lpips"])
  axis.set_title(title+title_suffix)

def interpolate(img_input, img_ground_truth, new_dimension, interpolation):
  img_new = cv2.resize(img_input, new_dimension, interpolation)
  img_psnr = psnr(img_new,img_ground_truth)
  img_ssim = ssim(img_new, img_ground_truth, multichannel=True)

  return img_new, img_psnr, img_ssim

def mean_dict_in_list(dict, key):
  i=0
  sum=0.0
  for data in dict:
    i=i+1
    sum = sum + data[key]
  return sum / i

def mean_metric(result,list):
  scores={}
  for score in list:
    scores[score] = mean_dict_in_list(result,score)
  return scores

def write_dataframe(selected_result):
  import pandas

  print(pandas.DataFrame(selected_result).T)
  #mean_scores = selected_result.mean(axis=1)
  #col_names=["SCORE","R-axis","G-axis","B-axis"]
  col_names=["SCORE"]
  row_names=["PSNR","MSE","SSIM"]
  #print(mean_scores)
  #print(pandas.DataFrame(mean_scores, row_names, col_names))

## Çözünürlük İyileştir

### Toplu İşlem - Çözünürlük İyileştirme

In [ ]:
#@title INTERPOLATIONS

if enhancement_mode:

  import matplotlib.pyplot as plt


  interpolation = None
  #Creating diff directories if not exists
  for method in Methods:
    dir = output_diff_dir[method]
    create_directory_if_not_exists(dir)

  count=0
  print("Input dir for INTERPOLATION is "+input_dir)
  for file in get_files(input_dir):
    count=count+1
    print(str(count)+"-"*10+"INTERPOLATING: "+file+"-"*10)
    img_ground_truth,_ = file_to_matrix(ground_truth_dir+file,Conversions.NONE, remove_alpha=True, normalize_factor=255.0)
    img,_ = file_to_matrix(input_dir+file,Conversions.NONE, remove_alpha=True, normalize_factor=255.0)


    #Çıktıya göre boyut ayarlamak yerine GİRDİYİ 4 İLE ÇARP!!!
    new_dimension=(img_ground_truth.shape[1],img_ground_truth.shape[0])



    print("NEAREST NEIGHTBORHOOD")
    img_new = cv2.resize(img, new_dimension, interpolation=cv2.INTER_NEAREST)
    show_stats(img_new)
    img_new = normalize_values(img_new,0,1)
    save_image(output_dir[Methods.INTERPOLATION_NN] +file, img_new)
    show_stats(img_new)

    print("LINEAR")
    img_new = cv2.resize(img, new_dimension, interpolation=cv2.INTER_LINEAR)
    show_stats(img_new)
    img_new = normalize_values(img_new,0,1)
    save_image(output_dir[Methods.INTERPOLATION_LINEAR] +file, img_new)
    show_stats(img_new)

    print("CUBIC")
    img_new = cv2.resize(img, new_dimension, interpolation=cv2.INTER_CUBIC)
    show_stats(img_new)
    img_new = normalize_values(img_new,0,1)
    save_image(output_dir[Methods.INTERPOLATION_CUBIC] +file, img_new)
    show_stats(img_new)

    print("LANCZOS")
    img_new = cv2.resize(img, new_dimension, interpolation=cv2.INTER_LANCZOS4)
    show_stats(img_new)
    img_new = normalize_values(img_new,0,1)
    save_image(output_dir[Methods.INTERPOLATION_LANCZOS] +file, img_new)
    show_stats(img_new)


In [ ]:
#@title SRGAN
if enhancement_mode:
  #Yeni görsel üretmek için output klasörünün temizlenmesi gerekiyor, eski resmin üstüne yazmıyor.
  os.chdir(base_dir[Methods.SRGAN])

  !python "{base_dir[Methods.SRGAN]}validate.py" -lr_dir "{input_dir}" -hr_dir "{ground_truth_dir}" -sr_dir "{output_dir[Methods.SRGAN]}"

  '''
  for file in filenames:
    print("File is "+file)
    !python "{base_dir[Methods.SRGAN]}validate.py" -file_name "{input_dir}{file}"
  '''

  #config.py içinde belirtlen datasete göre yapar
  #!python validate.py

#### Real-ESRGAN

In [ ]:
#@title Gereksinimleri Yükleme
if enhancement_mode:
  #Real-ESRGAN
  # Install basicsr - https://github.com/xinntao/BasicSR
  # We use BasicSR for both training and inference
  !pip install basicsr
  # facexlib and gfpgan are for face enhancement
  !pip install facexlib
  !pip install gfpgan
  !pip install ffmpeg-python
  !pip install -r requirements.txt
  !python setup.py develop

In [ ]:
#@title Çalıştırma
if enhancement_mode:
  os.chdir(base_dir[Methods.REAL_ESRGAN])

  filenames=get_files(input_dir)

  for file in filenames:
    print("File is "+file)
    !python "{base_dir[Methods.REAL_ESRGAN]}inference_realesrgan.py" -n RealESRGAN_x4plus -o "{output_dir[Methods.REAL_ESRGAN]}" --suffix "" -i "{input_dir}{file}"

#### Swin2SR

In [ ]:
#@title Gereksinimleri Yükleme
if enhancement_mode:
  !pip install timm
  !python -c "from timm import list_models; print(list_models(pretrained=True)[:5])"

In [ ]:
#@title Çalıştırma
if enhancement_mode:
  #Yeni görsel üretmek için output klasörünün temizlenmesi gerekiyor, eski resmin üstüne yazmıyor.
  os.chdir(base_dir[Methods.SWIN2SR])
  !python "{base_dir[Methods.SWIN2SR]}main_test_swin2sr.py" --task classical_sr --scale 4 --training_patch_size 64 --model_path model_zoo/swin2sr/Swin2SR_ClassicalSR_X4_64.pth --folder_lq "{input_dir}" --folder_gt "{ground_truth_dir}" --save_img_only

### İyileştirilmiş Görsel İnceleme

input olarak kullanılacak görseli keskinleştir. Ancak bunu yaparken RGB'nin tamamını düşünmek gerekecektir.

Aşağıdaki kaynaklar keskinleştirme konusunda fikir verecektir.

https://www.analyticsvidhya.com/blog/2021/08/sharpening-an-image-using-opencv-library-in-python/

https://stackoverflow.com/questions/60894593/how-to-sharpen-the-edges-in-opencv-python

https://towardsdatascience.com/image-processing-with-python-blurring-and-sharpening-for-beginners-3bcebec0583a

Daha iyi bir sharpen kernel mümkün mü? Yeni bir sharpen kernel geliştirilemez mi? Kenarları daha iyi ortaya çıkartan ancak yakın dokular üzerinde çok oynama yapmayan bir çözüm gerekiyor. Böylece doku detayları yakalanabilir.

**En uygun KERNEL işlemleri için fark görselleri üzerindeki mesafeye bakılması gerekecek. En uygununu seçmek için de tüm veriseti üzerinde çalıştırıp ortalama sonuçlar karşılaştırılmalı.**

SREdgeNet İNCELENMELİ

#### Edge Detection - old

In [ ]:
if (edge_detection_mode):

  # Modifiye edilmiş dosya kaydedilmeli ve sonra yolu gönderilmeli
  import cv2
  from google.colab.patches import cv2_imshow

  file_name = "butterfly.png"
  file_path = "/content/drive/MyDrive/Colab Notebooks/SRGAN/dataset/Set5/LRbicx4/"
  modified_file_path = "/content/drive/MyDrive/Colab Notebooks/SRGAN/temp.png"

  image = cv2.imread(file_path+file_name, cv2.IMREAD_UNCHANGED).astype(np.float32)
  cv2_imshow(image)
  #image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


  kernel = []

  #Sobel–Feldman operator, Alternative, https://en.wikipedia.org/wiki/Sobel_operator
  kernel.append(
                np.array([[3,    0,    -3],
                          [10,    0,    -10],
                          [3,    0,    -3]])
  )

  #Sharpen
  kernel.append(
                np.array([[0, -1, 0],
                          [-1, 5,-1],
                          [0, -1, 0]])/8
  )

  kernel.append(
                np.array([[1,0,-1],
                          [2,0,-2],
                          [1,0,-1]])/10
  )

  for i in range(len(kernel)):
    image = cv2.filter2D(src=image, ddepth=-1, kernel=kernel[i])
    print("Kernel: "+str(i))
    cv2_imshow(image)

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cv2_imshow(gray_image)

    gray_image = cv2.filter2D(src=gray_image, ddepth=-1, kernel=kernel[i])
    cv2_imshow(gray_image)


    gray_image = cv2.filter2D(src=gray_image, ddepth=-1, kernel=kernel[i])
    cv2_imshow(gray_image)





    continue
    result = cv2.imwrite(modified_file_path, image)

    if result:
      modified_file_path_space_out = modified_file_path.replace(" ","@")
      !python validate.py -file_name "butterfly.png" -modified_file_path $modified_file_path_space_out

      show_compared_images(input_dir+"/"+file_name,output_dir+"/"+file_name,ground_truth_dir+"/"+file_name)
    else:
      print("File write error")

    # Üzerinde oynanmış görseli kabul edecek bir yapı düşünmek gerekecek, dosya isminin yanında bu veri de gönderilmeli
    # Sadece STRING kabul ediliyor. Dosya sistemine yazmak dışında bir seçenek yok gibi...

#### Sharpen Kernel and Prewitt - BAŞARILI GÖRÜNÜYOR

In [ ]:
def pixel_extend(img,x,y,value,extend_size):
  for i in range(extend_size):
    for j in range(extend_size):
      img[i+x][j+y]=value


def apply_local_filter(img,ref_img,x,y,filter,extend_size):
  if (x - filter.shape[0]/2 < 0 or y - filter.shape[1]/2 < 0):
    return
  if (x + filter.shape[0]/2 + extend_size >= img.shape[0]-2 or y + filter.shape[1]/2 + extend_size >= img.shape[1]-2):
    return

  filter_width = filter.shape[0]
  filter_height =  filter.shape[1]

  start_x = x - int(filter_width/2)
  start_y = y - int(filter_height/2)

  end_x = x + filter_width - 1
  end_y = y + filter_height - 1
  for plus_i in range(extend_size):

    for i in range(start_x + plus_i,end_x + plus_i):
      filter_i=0
      for plus_j in range(extend_size):
        filter_j=0
        for j in range(start_y + plus_j,end_y + plus_j):
          #print(str(i)+"-"+str(j) + "FILTER" + str(filter_i)+"-"+str(filter_j))
          img[i][j] = ref_img[i][j] * filter[filter_i][filter_j]
          filter_j=filter_j+1
      filter_i=filter_i+1

#ref_img=img.copy()
#apply_local_filter(img,ref_img,50,50,filter,4)

In [ ]:
if (edge_detection_mode):

  # 1-Grayscale
  # 2-Sharpen kernel 1. geçiş
  # 3-Sharpen kernel 2. geçiş
  import cv2
  from google.colab.patches import cv2_imshow

  show_images=True

  create_directory_if_not_exists(input_edge_dir)
  create_directory_if_not_exists(output_modified_dir[Methods.SRGAN])

  file_names = ["butterfly.png"]

  #If working with only 1 file, comment the below line
  #file_names = get_files(input_dir)


  for file_name in file_names:
    print(file_name)

    modified_file_path = input_edge_dir+file_name

    print(input_dir+file_name)

    image,_ = file_to_matrix(input_dir+file_name,Conversions.NONE)
    #image = image[...,::-1]
    #cv2_show_image(image)


    #image = cv2.imread(input_dir+file_name, cv2.IMREAD_UNCHANGED).astype(np.float32)
    if show_images:
      print("4x zoom")
      cv2_show_image(image)

    #image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


    kernel = []
    #Sharpen
    kernel.append(
                  np.array([[0, -1, 0],
                            [-1, 5,-1],
                            [0, -1, 0]])/6
    )


    kernel.append(
                  np.array([[0, -1, 0],
                            [-1, 5,-1],
                            [0, -1, 0]])/6
    )

    print(np.array(kernel).shape)

    #print("Kernel pass on original low-res image:")
    #image = cv2.filter2D(src=image, ddepth=-1, kernel=kernel[i])
    #cv2_imshow(image)


    #BGR2GRAY OR RGB2GRAY?
    #gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    if show_images:
      print("Grayscale")
      cv2_show_image(gray_image,4,False)

    for i in range(len(kernel)):
      gray_image = cv2.filter2D(src=gray_image, ddepth=-1, kernel=kernel[i])
      if show_images:
        print("After pass of kernel: "+str(i))
        cv2_show_image(gray_image,4,False)

    #Kaldırılacak
    #(thresh, black_white_image) = cv2.threshold(gray_image, 5, 255, cv2.THRESH_BINARY)

    #print("Black and White")
    #cv2_show_image(black_white_image)

    #result = cv2.imwrite(modified_file_path, black_white_image)

    '''
    #CANNY
    gray_image = np.uint8(gray_image)
    gray_image = cv2.Canny(gray_image,100,200)
    gray_image = cv2.Canny(image=gray_image,threshold1=100,threshold2=200)
    cv2_show_image(gray_image)
    '''



    #PREWITT YİNE KULLANILSIN
    kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])*0.5
    kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])*0.5
    img_prewittx = cv2.filter2D(gray_image, -1, kernelx)
    img_prewitty = cv2.filter2D(gray_image, -1, kernely)

    img_prewitt = img_prewittx + img_prewitty


    print("Prewitt Applied #1")
    cv2_show_image(img_prewitt,4,False)


    result = cv2.imwrite(modified_file_path, img_prewitt)
    #print(img_prewitt.round(decimals=0))

    img = img_prewitt
    #img = gray_image


    '''
    #Example of bad idea :)
    #0 and 255 polarizing according to mean-std
    threshold = img.mean() + img.std()
    for i in range(img.shape[0]):
      for j in range(img.shape[1]):
        if (img[i][j]>threshold):
          img[i][j]=255
        else:
          img[i][j]=0
    print("Trimmed values out of threshold")
    cv2_show_image(img,4,False)
    '''

    #Dosyaya yazılan ve dosyadan okunan görseller farklı görünüyor!!!

    #img = cv2.imread(modified_file_path)
    #cv2_show_image(img)
    #print(img)
    #img.describe()
    #print("Reading from file after saving:")
    show_stats(img)


    #ROI'deki pikselin fark değerine göre daha güçlü bir şekilde keskinleştirme uygulanabilir.
    #Yüksek fark = yüksek keskinleştirme

    '''

    filter=np.array([[1, 1.1, 1],
                    [1.1, 1.5, 1.1],
                    [1, 1.1, 1]])
    extend_size=1

    enhanced_file=output_dir[Methods.SRGAN]+file_name
    print(enhanced_file)
    enhanced_img,_=file_to_matrix(enhanced_file,Conversions.NONE)
    enhanced_ref_img = enhanced_img.copy()
    enhanced_spot_img = enhanced_img.copy()
    for i in range(img.shape[0]):
      for j in range(img.shape[1]):
        if (img[i][j]>0):
          value=[255.,0.,0,]
          pixel_extend(enhanced_spot_img,i*4,j*4,value,4)
          apply_local_filter(enhanced_img,enhanced_ref_img,i*4,j*4,filter,extend_size)
          #enhanced_img[i][j]=value

    cv2_show_image(enhanced_spot_img)
    cv2_show_image(enhanced_img)
    cv2_show_image(enhanced_ref_img)

    show_stats(enhanced_img)
    enhanced_img = normalize_values(enhanced_img,0,1)
    save_image(output_modified_dir[Methods.SRGAN]+file_name, enhanced_img)

    '''

In [ ]:
#@title Edge Detectors (Gradiant Magnitude?)
if (edge_detection_mode):

  # https://gist.github.com/rahit/c078cabc0a48f2570028bff397a9e154
  import cv2
  from google.colab.patches import cv2_imshow
  import numpy as np

  create_directory_if_not_exists(input_edge_dir)
  file_name = "butterfly.png"


  file_names=get_files(input_dir)
  for file_name in file_names:
    print(file_name)

    modified_file_path = input_edge_dir+file_name





    img = cv2.imread(input_dir+file_name)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gaussian = cv2.GaussianBlur(gray,(3,3),0)

    #canny
    img_canny = cv2.Canny(img,100,200)

    #sobel
    img_sobelx = cv2.Sobel(img_gaussian,cv2.CV_8U,1,0,ksize=5)
    img_sobely = cv2.Sobel(img_gaussian,cv2.CV_8U,0,1,ksize=5)
    img_sobel = img_sobelx + img_sobely


    #prewitt
    kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
    kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
    img_prewittx = cv2.filter2D(img_gaussian, -1, kernelx)
    img_prewitty = cv2.filter2D(img_gaussian, -1, kernely)


    print("Original Image")
    cv2_imshow(img)

    print("Canny")
    cv2_imshow(img_canny)

    #print("Sobel X")
    #cv2_imshow(img_sobelx)

    #print("Sobel Y")
    #cv2_imshow(img_sobely)

    print("Sobel")
    cv2_imshow(img_sobel)

    #print("Prewitt X")
    #cv2_imshow(img_prewittx)

    #print("Prewitt Y")
    #cv2_imshow(img_prewitty)

    print("Prewitt")
    cv2_imshow(img_prewittx + img_prewitty)


    #result = cv2.imwrite(modified_file_path, black_white_image)


  if False:
    if result:
      modified_file_path_space_out = modified_file_path.replace(" ","@")
      !python validate.py -file_name $file_name -modified_file_path $modified_file_path_space_out

      show_compared_images(input_dir+"/"+file_name,output_dir+"/"+file_name,ground_truth_dir+"/"+file_name)
    else:
      print("File write error")

  # Görseller çift kutuplu bir yapıya dönüştürülmeli

#### Edge PSNR ve SSIM Hesaplama

In [ ]:
if (edge_detection_mode):

  #ŞİMDİLİK BU KISIM ÖNEMLİ DEĞİL, EXTRACTED EDGE YETERİNCE İYİ SONUÇ VERMEZSE BAKILABİLİR.

  input_edge_files = get_files(input_edge_dir)

  overall_result=[]

  def edge_assessment(output_diff_dir):
    overall_result=[]
    for file_name in input_edge_files:
      input_file_name = input_edge_dir + file_name
      output_file_name = output_diff_dir + file_name
      #ground_truth_file_name = ground_truth_dir + file_name

      input_matrix = file_to_matrix(input_file_name,Conversion.NONE,True,255.0)
      output_matrix = file_to_matrix(output_file_name,Conversion.NONE,True,255.0)
      scores = compare_images(cv2.resize(input_matrix, (input_matrix.shape[1]*4,input_matrix.shape[0]*4), cv2.INTER_NEAREST),output_matrix)
      overall_result.append(scores)

  #edge_assessment(output_diff_dir["srgan"])

  print("-"*10 + " MEAN OF EDGE SCORES " + "-"*10)

  import pandas

  overall_result = np.array(overall_result)

  mean_scores = overall_result.mean(axis=0)
  col_names=["SCORE","R-axis","G-axis","B-axis"]
  row_names=["PSNR","MSE","SSIM"]
  print(pandas.DataFrame(mean_scores, row_names, col_names))



## Sonuçları Göster

In [ ]:
#DISTS hack
%cd '/content/drive/My Drive/Colab Notebooks/SRGAN/7-DISTS-IQA'

#os.chdir(base_dir[Methods.SWIN2SR])

if (show_comparison_results):
  import numpy as np


  filenames = get_files(input_dir)
  overall_result={}

  calculate_all_dataset=True # If False, only show_images_list will be calculated.
  show_images_list=["baby.png","bird.png","butterfly.png","head.png","woman.png","ppt3.png","236037.png","img_008.png"]
  show_images_list=["baboon.png","lenna.png"]

  show_all_images=False # If True, all images will be shown, else only specified images will be shown
  show_process_results=False
  save_diff=True # If False, GT will be placed at the end of the image list

  methods_list = Methods
  conversions_list = Conversions



  #methods_list = [Methods.INTERPOLATION_CUBIC]
  #conversions_list = [Conversions.NONE]


  for method in methods_list:
    overall_result[method.name]={}
    for conversion in conversions_list:
      overall_result[method.name][conversion.name]=[]

  count=0
  for file in filenames:
    if (file in show_images_list):
      show_image=True
    else:
      if (not calculate_all_dataset):
        continue
      show_image=show_all_images
    count=count+1
    print("File: #"+str(count)+": "+file)
    for conversion in conversions_list:
      print("CONVERSION: "+conversion.name)
      show_compared_images(input_dir, ground_truth_dir,file,overall_result, conversion, methods=methods_list, save_diff=save_diff, show_image=show_image,modified_output=modified_results)


  if (show_process_results):
    print("-"*10 + " RESULTS OF EACH PROCESS" + "-"*10)
    pretty_print(overall_result)

if (show_comparison_results):

  print("-"*10 + " MEAN OF OVERALL SCORES" + "-"*10)

  overall_mean={}
  scores_list = ["psnr","ssim","dists","adists","lpips"]
  for method in methods_list:
    for conversion in conversions_list:
      name = method.name.ljust(25)+conversion.name.ljust(10)
      overall_mean[name]={}

      overall_mean[name]=mean_metric(overall_result[method.name][conversion.name],scores_list)

  #pretty_print(overall_mean)
  write_dataframe(overall_mean)
